In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shrey_shah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shrey_shah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [4]:
os.getcwd()

'/Users/shrey_shah/Desktop/Text_Mining'

In [5]:
initial_df = pd.read_csv(os.getcwd()+"/Dataset/train.csv")

In [6]:
initial_df.head(5)

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,0,de_0203609,product_de_0865382,reviewer_de_0267719,1,Armband ist leider nach 1 Jahr kaputt gegangen,Leider nach 1 Jahr kaputt,de,sports
1,1,de_0559494,product_de_0678997,reviewer_de_0783625,1,In der Lieferung war nur Ein Akku!,EINS statt ZWEI Akkus!!!,de,home_improvement
2,2,de_0238777,product_de_0372235,reviewer_de_0911426,1,"Ein Stern, weil gar keine geht nicht. Es hande...",Achtung Abzocke,de,drugstore
3,3,de_0477884,product_de_0719501,reviewer_de_0836478,1,"Dachte, das wären einfach etwas festere Binden...",Zu viel des Guten,de,drugstore
4,4,de_0270868,product_de_0022613,reviewer_de_0736276,1,Meine Kinder haben kaum damit gespielt und nac...,Qualität sehr schlecht,de,toy


In [7]:
english_lang_train_reviews = initial_df[initial_df['language'] == 'en']
english_lang_train_reviews.drop(columns=['Unnamed: 0', 'reviewer_id'], inplace=True)
english_lang_train_reviews.head()

/var/folders/j2/sk_tpq311dn0rsb2ksfpkzb00000gn/T/ipykernel_15424/2487562124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_lang_train_reviews.drop(columns=['Unnamed: 0', 'reviewer_id'], inplace=True)


,review_id,product_id,stars,review_body,review_title,language,product_category
200000,en_0964290,product_en_0740675,1,Arrived broken. Manufacturer defect. Two of th...,I'll spend twice the amount of time boxing up ...,en,furniture
200001,en_0690095,product_en_0440378,1,the cabinet dot were all detached from backing...,Not use able,en,home_improvement
200002,en_0311558,product_en_0399702,1,I received my first order of this product and ...,The product is junk.,en,home
200003,en_0044972,product_en_0444063,1,This product is a piece of shit. Do not buy. D...,Fucking waste of money,en,wireless
200004,en_0784379,product_en_0139353,1,went through 3 in one day doesn't fit correct ...,bubble,en,pc


In [8]:
english_lang_train_reviews.loc[200003, ('review_body')]

"This product is a piece of shit. Do not buy. Doesn't work, and then I try to call for customer support, it won't take my number. Fucking rip off!"

In [9]:
english_lang_train_reviews.shape

(200000, 7)

In [10]:
german_lang_train_reviews = initial_df[initial_df['language'] == 'de']
german_lang_train_reviews.drop(columns=['Unnamed: 0', 'reviewer_id'], inplace=True)
german_lang_train_reviews.head()

/var/folders/j2/sk_tpq311dn0rsb2ksfpkzb00000gn/T/ipykernel_15424/1976325618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  german_lang_train_reviews.drop(columns=['Unnamed: 0', 'reviewer_id'], inplace=True)


,review_id,product_id,stars,review_body,review_title,language,product_category
0,de_0203609,product_de_0865382,1,Armband ist leider nach 1 Jahr kaputt gegangen,Leider nach 1 Jahr kaputt,de,sports
1,de_0559494,product_de_0678997,1,In der Lieferung war nur Ein Akku!,EINS statt ZWEI Akkus!!!,de,home_improvement
2,de_0238777,product_de_0372235,1,"Ein Stern, weil gar keine geht nicht. Es hande...",Achtung Abzocke,de,drugstore
3,de_0477884,product_de_0719501,1,"Dachte, das wären einfach etwas festere Binden...",Zu viel des Guten,de,drugstore
4,de_0270868,product_de_0022613,1,Meine Kinder haben kaum damit gespielt und nac...,Qualität sehr schlecht,de,toy


In [11]:
german_lang_train_reviews.shape

(200000, 7)

In [12]:
german_lang_train_reviews.to_csv('german_lang_train_reviews.csv', index=False)

In [13]:
english_lang_train_reviews.to_csv('english_lang_train_reviews.csv', index=False)

In [14]:
test_df = pd.read_csv(os.getcwd()+"/Dataset/test.csv")

In [15]:
test_df.head()

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,0,de_0784695,product_de_0572654,reviewer_de_0645436,1,"Leider, leider nach einmal waschen ausgebliche...",Leider nicht zu empfehlen,de,home
1,1,de_0759207,product_de_0567331,reviewer_de_0183703,1,zunächst macht der Anker Halter einen soliden ...,Gummierung nach 6 Monaten kaputt,de,wireless
2,2,de_0711785,product_de_0482105,reviewer_de_0182152,1,Siegel sowie Verpackung war beschädigt und war...,Flohmarkt ware,de,industrial_supplies
3,3,de_0964430,product_de_0616480,reviewer_de_0991563,1,Habe dieses Produkt NIE erhalten und das Geld ...,Katastrophe,de,industrial_supplies
4,4,de_0474538,product_de_0228702,reviewer_de_0316188,1,Die Träger sind schnell abgerissen,Reißverschluss klemmt,de,luggage


In [16]:
english_lang_test_reviews = test_df[test_df['language'] == 'en']
english_lang_test_reviews.drop(columns=['Unnamed: 0', 'reviewer_id'], inplace=True)
english_lang_test_reviews.head()

/var/folders/j2/sk_tpq311dn0rsb2ksfpkzb00000gn/T/ipykernel_15424/3101386127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_lang_test_reviews.drop(columns=['Unnamed: 0', 'reviewer_id'], inplace=True)


,review_id,product_id,stars,review_body,review_title,language,product_category
5000,en_0199937,product_en_0902516,1,"These are AWFUL. They are see through, the fab...",Don’t waste your time!,en,apparel
5001,en_0863335,product_en_0348072,1,I bought 4 and NONE of them worked. Yes I used...,One Star,en,other
5002,en_0565010,product_en_0356154,1,On first use it didn't heat up and now it does...,Totally useless,en,other
5003,en_0963290,product_en_0583322,1,You want an HONEST answer? I just returned fro...,Gold filled earrings,en,jewelry
5004,en_0238156,product_en_0487636,1,The glue works fine but the container is impos...,Poor container,en,industrial_supplies


In [17]:
german_lang_test_reviews = test_df[test_df['language'] == 'de']
german_lang_test_reviews.drop(columns=['Unnamed: 0', 'reviewer_id'], inplace=True)
german_lang_test_reviews.head()

/var/folders/j2/sk_tpq311dn0rsb2ksfpkzb00000gn/T/ipykernel_15424/3435924005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  german_lang_test_reviews.drop(columns=['Unnamed: 0', 'reviewer_id'], inplace=True)


,review_id,product_id,stars,review_body,review_title,language,product_category
0,de_0784695,product_de_0572654,1,"Leider, leider nach einmal waschen ausgebliche...",Leider nicht zu empfehlen,de,home
1,de_0759207,product_de_0567331,1,zunächst macht der Anker Halter einen soliden ...,Gummierung nach 6 Monaten kaputt,de,wireless
2,de_0711785,product_de_0482105,1,Siegel sowie Verpackung war beschädigt und war...,Flohmarkt ware,de,industrial_supplies
3,de_0964430,product_de_0616480,1,Habe dieses Produkt NIE erhalten und das Geld ...,Katastrophe,de,industrial_supplies
4,de_0474538,product_de_0228702,1,Die Träger sind schnell abgerissen,Reißverschluss klemmt,de,luggage


In [18]:
german_lang_test_reviews.to_csv('german_lang_test_reviews.csv', index=False)

In [19]:
english_lang_test_reviews.to_csv('english_lang_test_reviews.csv', index=False)

In [20]:
english_lang_train_reviews['product_category'].value_counts()

home                        17679
apparel                     15951
wireless                    15717
other                       13418
beauty                      12091
drugstore                   11730
kitchen                     10382
toy                          8745
sports                       8277
automotive                   7506
lawn_and_garden              7327
home_improvement             7136
pet_products                 7082
digital_ebook_purchase       6749
pc                           6401
electronics                  6186
office_product               5521
shoes                        5197
grocery                      4730
book                         3756
baby_product                 3150
furniture                    2984
jewelry                      2747
camera                       2139
industrial_supplies          1994
digital_video_download       1364
luggage                      1328
musical_instruments          1102
video_games                   775
watch         

In [21]:
class TextPreprocessor():

    def __init__(self):

        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))


    def preprocess_text(self, text):

        text = text.lower() # Convert to lower case
        text = re.sub(r'<.*?>', '', text) # Remove punctuations
        text = re.sub(r'[^\w\s]', '', text) # Remove HTML tags
        tokens = nltk.word_tokenize(text) # Tokenize the text
        tokens = [self.stemmer.stem(token) for token in tokens if token not in self.stop_words] # Stemming the text
        return ' '.join(tokens)

In [22]:
preprocessor = TextPreprocessor()

# Apply the custom transformer to a specific column and create a new column
english_lang_train_reviews['preprocessed_text'] = english_lang_train_reviews['review_body'].apply(preprocessor.preprocess_text)
english_lang_test_reviews['preprocessed_text'] = english_lang_test_reviews['review_body'].apply(preprocessor.preprocess_text)

/var/folders/j2/sk_tpq311dn0rsb2ksfpkzb00000gn/T/ipykernel_15424/1948113571.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_lang_train_reviews['preprocessed_text'] = english_lang_train_reviews['review_body'].apply(preprocessor.preprocess_text)
/var/folders/j2/sk_tpq311dn0rsb2ksfpkzb00000gn/T/ipykernel_15424/1948113571.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_lang_test_reviews['preprocessed_text'] = english_lang_test_reviews['review_body'].apply(preprocessor.preprocess_text)


NameError: name 'english_train_reviews' is not defined

In [24]:
english_lang_train_reviews.head(5)

,review_id,product_id,stars,review_body,review_title,language,product_category,preprocessed_text
200000,en_0964290,product_en_0740675,1,Arrived broken. Manufacturer defect. Two of th...,I'll spend twice the amount of time boxing up ...,en,furniture,arriv broken manufactur defect two leg base co...
200001,en_0690095,product_en_0440378,1,the cabinet dot were all detached from backing...,Not use able,en,home_improvement,cabinet dot detach back got
200002,en_0311558,product_en_0399702,1,I received my first order of this product and ...,The product is junk.,en,home,receiv first order product broke order second ...
200003,en_0044972,product_en_0444063,1,This product is a piece of shit. Do not buy. D...,Fucking waste of money,en,wireless,product piec shit buy doesnt work tri call cus...
200004,en_0784379,product_en_0139353,1,went through 3 in one day doesn't fit correct ...,bubble,en,pc,went 3 one day doesnt fit correct couldnt get ...


## Using Count Vectorizer

In [26]:
# Create a Count Vectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,1))

# Transform the preprocessed text data into a document-term matrix
count_matrix = count_vectorizer.fit_transform(english_lang_train_reviews['preprocessed_text'])

In [32]:
#Count-based feature extraction
from sklearn.feature_extraction.text import CountVectorizer

train_text = english_lang_train_reviews["preprocessed_text"]
test_text = english_lang_test_reviews["preprocessed_text"]

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
train_data_cv = vectorizer.fit_transform(train_text)

# observe the words in the created dictionary across the document
pd.set_option('display.width', 1000)
print(len(vectorizer.vocabulary_), " ... ", list(vectorizer.vocabulary_.items())[0:100],"\n")

test_data_cv = vectorizer.transform(test_text)

print(test_data_cv.shape)

44263  ...  [('arriv', 3560), ('broken', 6353), ('manufactur', 23327), ('defect', 10825), ('two', 40239), ('leg', 21835), ('base', 4506), ('complet', 9018), ('form', 15386), ('way', 42341), ('insert', 19713), ('caster', 7458), ('unpackag', 40832), ('entir', 13304), ('chair', 7673), ('hardwar', 17792), ('notic', 26247), ('ill', 19223), ('spend', 36125), ('twice', 40199), ('amount', 2916), ('time', 39068), ('box', 6003), ('whole', 42941), ('useless', 41250), ('thing', 38684), ('send', 33931), ('back', 4106), ('1star', 562), ('review', 32246), ('part', 28056), ('never', 25602), ('got', 16812), ('sit', 34881), ('go', 16588), ('far', 14192), ('includ', 19426), ('pictur', 28808), ('inject', 19635), ('mold', 24615), ('qualiti', 30813), ('assur', 3718), ('process', 30155), ('miss', 24431), ('though', 38833), ('hesit', 18299), ('buy', 6813), ('make', 23209), ('wonder', 43287), ('arent', 3465), ('structur', 37096), ('support', 37494), ('dont', 12013), ('imped', 19308), ('assembl', 3685), ('cabin

In [33]:
X_train = train_data_cv
X_test = test_data_cv
y_train = english_lang_train_reviews['product_category']
y_test = english_lang_test_reviews['product_category']

In [35]:
lr = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=50000)
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)

print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))

print(y_test + '\t' + predictions)

Accuracy score:  0.4608
Precision score:  0.46043675135409373
Recall score:  0.4608
F1 score:  0.44933555276736753
5000                apparel\tapparel
5001                    other\tother
5002                   other\tbeauty
5003                  jewelry\tother
5004    industrial_supplies\tkitchen
                    ...             
9995            drugstore\tdrugstore
9996                  beauty\tbeauty
9997                      home\thome
9998          automotive\tautomotive
9999                   apparel\thome
Name: product_category, Length: 5000, dtype: object


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
report = classification_report(y_test, predictions)
print(report)

                          precision    recall  f1-score   support

                 apparel       0.51      0.78      0.62       390
              automotive       0.50      0.44      0.47       187
            baby_product       0.37      0.22      0.28        77
                  beauty       0.46      0.50      0.48       301
                    book       0.57      0.39      0.46       105
                  camera       0.32      0.13      0.18        54
  digital_ebook_purchase       0.82      0.87      0.85       173
  digital_video_download       0.67      0.58      0.62        38
               drugstore       0.38      0.41      0.40       289
             electronics       0.37      0.21      0.27       153
               furniture       0.58      0.40      0.48        62
                 grocery       0.50      0.57      0.54       115
                    home       0.33      0.47      0.39       440
        home_improvement       0.40      0.33      0.36       183
     indu

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Using TF-IDF

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed text data into TF-IDF vectors
tfidf_train = tfidf_vectorizer.fit_transform(train_text)
tfidf_test = tfidf_vectorizer.transform(test_text)

In [42]:
tfidf_train.shape

(200000, 44263)

In [45]:
lr_tfidf = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=50000)
lr_tfidf.fit(tfidf_train, y_train)
predictions = lr_tfidf.predict(X_test)

print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))

print(y_test, '\t', predictions)

Accuracy score:  0.4622
Precision score:  0.4645109147407582
Recall score:  0.4622
F1 score:  0.44823348750575515
5000                apparel
5001                  other
5002                  other
5003                jewelry
5004    industrial_supplies
               ...         
9995              drugstore
9996                 beauty
9997                   home
9998             automotive
9999                apparel
Name: product_category, Length: 5000, dtype: object 	 ['apparel' 'electronics' 'beauty' ... 'home' 'automotive' 'home']


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
report = classification_report(y_test, predictions)
print(report)

                          precision    recall  f1-score   support

                 apparel       0.55      0.75      0.64       390
              automotive       0.46      0.46      0.46       187
            baby_product       0.23      0.38      0.28        77
                  beauty       0.44      0.55      0.49       301
                    book       0.67      0.38      0.48       105
                  camera       0.33      0.37      0.35        54
  digital_ebook_purchase       0.75      0.90      0.82       173
  digital_video_download       0.68      0.50      0.58        38
               drugstore       0.48      0.37      0.41       289
             electronics       0.32      0.34      0.33       153
               furniture       0.28      0.66      0.39        62
                 grocery       0.52      0.62      0.57       115
                    home       0.44      0.30      0.36       440
        home_improvement       0.37      0.42      0.39       183
     indu

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
class TextPreprocessor_no_lemmatization():

    def __init__(self):
        
        self.stop_words = set(stopwords.words('english'))


    def preprocess_text(self, text):

        text = text.lower() # Convert to lower case
        text = re.sub(r'<.*?>', '', text) # Remove punctuations
        text = re.sub(r'[^\w\s]', '', text) # Remove HTML tags
        tokens = nltk.word_tokenize(text) # Tokenize the text
        filtered_words = [token for token in tokens if token not in self.stop_words] # Remove stop words
        return ' '.join(filtered_words)

In [55]:
preprocessor = TextPreprocessor_no_lemmatization()

# Apply the custom transformer to a specific column and create a new column
english_lang_train_reviews['preprocessed_text_no_lemmatization'] = english_lang_train_reviews['review_body'].apply(preprocessor.preprocess_text)
english_lang_test_reviews['preprocessed_text_no_lemmatization'] = english_lang_test_reviews['review_body'].apply(preprocessor.preprocess_text)

/var/folders/j2/sk_tpq311dn0rsb2ksfpkzb00000gn/T/ipykernel_15424/3396159977.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_lang_train_reviews['preprocessed_text_no_lemmatization'] = english_lang_train_reviews['review_body'].apply(preprocessor.preprocess_text)
/var/folders/j2/sk_tpq311dn0rsb2ksfpkzb00000gn/T/ipykernel_15424/3396159977.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_lang_test_reviews['preprocessed_text_no_lemmatization'] = english_lang_test_reviews['review_body'].appl

### Count Vectorizer

In [56]:
#Count-based feature extraction
from sklearn.feature_extraction.text import CountVectorizer

train_text = english_lang_train_reviews["preprocessed_text_no_lemmatization"]
test_text = english_lang_test_reviews["preprocessed_text_no_lemmatization"]

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
train_data_cv = vectorizer.fit_transform(train_text)

# observe the words in the created dictionary across the document
pd.set_option('display.width', 1000)
print(len(vectorizer.vocabulary_), " ... ", list(vectorizer.vocabulary_.items())[0:100],"\n")

test_data_cv = vectorizer.transform(test_text)

print(test_data_cv.shape)

60554  ...  [('arrived', 4408), ('broken', 8155), ('manufacturer', 31991), ('defect', 14629), ('two', 55493), ('legs', 30079), ('base', 5727), ('completely', 11880), ('formed', 21365), ('way', 58228), ('insert', 27188), ('casters', 9664), ('unpackaged', 56243), ('entire', 18371), ('chair', 9973), ('hardware', 24500), ('noticing', 35797), ('ill', 26378), ('spend', 49739), ('twice', 55440), ('amount', 3531), ('time', 53899), ('boxing', 7715), ('whole', 58941), ('useless', 56777), ('thing', 53401), ('send', 46727), ('back', 5191), ('1star', 611), ('review', 44424), ('part', 38132), ('never', 35041), ('got', 23215), ('sit', 48048), ('go', 22952), ('far', 19689), ('include', 26705), ('picture', 39179), ('injection', 27069), ('molding', 33744), ('quality', 42020), ('assurance', 4647), ('process', 41062), ('missed', 33464), ('though', 53574), ('hesitant', 25156), ('buy', 8785), ('makes', 31821), ('wonder', 59394), ('arent', 4274), ('missing', 33474), ('structures', 51185), ('supports', 51776)

In [57]:
X_train = train_data_cv
X_test = test_data_cv
y_train = english_lang_train_reviews['product_category']
y_test = english_lang_test_reviews['product_category']

In [58]:
lr = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=50000)
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)

print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))

print(y_test + '\t' + predictions)

Accuracy score:  0.4556
Precision score:  0.45232881092702676
Recall score:  0.4556
F1 score:  0.445314529138051
5000              apparel\tapparel
5001       other\thome_improvement
5002                 other\tbeauty
5003              jewelry\tjewelry
5004    industrial_supplies\tother
                   ...            
9995          drugstore\tdrugstore
9996                beauty\tbeauty
9997                    home\thome
9998        automotive\tautomotive
9999                 apparel\thome
Name: product_category, Length: 5000, dtype: object


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
